In [1]:
base_path = '/tf/PatchCL-MedSeg-jiyu'
dataset_path = '/tf/dataset/0_data_dataset_voc_950_kidney'
output_dir = base_path + '/dataset/splits/kidney'

supervised_loss_path = base_path + '/supervised pre training_loss.csv'
SSL_loss_path = base_path + '/SSL_loss.csv'
base_contrastive_model = base_path + "/best_contrast_size128.pth"
voc_mask_color_map = [
    [0, 0, 0], #_background
    [128, 0, 0] #kidney
]

In [2]:
import os
import sys
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import math
import time
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms

sys.path.append(base_path)

from utils.transform import Transform
from utils.stochastic_approx import StochasticApprox
from utils.model import Network
from utils.datasets_PASCAL import PascalVOCDataset
from utils.queues import Embedding_Queues
from utils.CELOSS import CE_loss
from utils.patch_utils import _get_patches
from utils.aug_utils import batch_augment
from utils.get_embds import get_embeddings
from utils.const_reg import consistency_cost
from utils.plg_loss import PCGJCL
from utils.torch_poly_lr_decay import PolynomialLRDecay
from utils.loss_file import save_loss

In [3]:
contrastive_batch_size = 128
img_size = 256
batch_size = 8
num_classes = 2
transform = Transform(img_size, num_classes)
embd_queues = Embedding_Queues(num_classes)
stochastic_approx = StochasticApprox(num_classes,0.5,0.8)
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
model = torch.load(base_contrastive_model)

In [11]:
labeled_dataset = PascalVOCDataset(txt_file=output_dir + "/1-3/labeled.txt", image_size=img_size, root_dir=dataset_path, labeled=True, colormap=voc_mask_color_map)
unlabeled_dataset = PascalVOCDataset(txt_file=output_dir + "/1-3/unlabeled.txt", image_size=img_size, root_dir=dataset_path, labeled=False, colormap=voc_mask_color_map)
val_dataset = PascalVOCDataset(txt_file=output_dir + "/val.txt", image_size=img_size, root_dir=dataset_path, labeled=True, colormap=voc_mask_color_map)

labeled_loader = DataLoader(labeled_dataset, batch_size=batch_size)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

print('number of labeled_dataset: ', len(labeled_dataset))
print('number of unlabeled_dataset: ', len(unlabeled_dataset))
print('number of val_dataset: ', len(val_dataset))

number of labeled_dataset:  285
number of unlabeled_dataset:  570
number of val_dataset:  95


In [16]:
# 測試數據加載器
for images, masks in val_loader:
    print(images.shape, masks.shape)
    break

torch.Size([8, 3, 256, 256]) torch.Size([8, 256, 256])


In [18]:
multi_channel_masks = torch.zeros((masks.size(0), 2, masks.size(1), masks.size(2)), dtype=torch.float32)
# 將單通道遮罩轉換為多通道遮罩
multi_channel_masks[:, 0, :, :] = masks.float()  # 原始遮罩
multi_channel_masks[:, 1, :, :] = 1 - masks.float()  # 反轉遮罩

print('multi_channel_masks shape: ', multi_channel_masks.shape)

multi_channel_masks shape:  torch.Size([8, 2, 256, 256])


In [20]:
#calculate supervied loss
images, multi_channel_masks = images.to(dev), multi_channel_masks.to(dev)
out = model(images)
print('masks shape: ', multi_channel_masks.shape)
print('output shape: ', out.shape)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


masks shape:  torch.Size([8, 2, 256, 256])
output shape:  torch.Size([8, 4, 256, 256])
